## Imports

In [ ]:
from sklearn import preprocessing, decomposition, cluster
from matplotlib import pyplot as plt
from collections import namedtuple
import pandas as pd
import numpy as np

# Load Dataset
Load the Million Song Dataset from 'YearPredictionsMSP.txt' (expected to be in the data folder)  
Dataset Source: https://archive.ics.uci.edu/ml/datasets/YearPredictionMSD  

The following information is from the UCI webpage for this dataset (linked above):

## Data Set Information:
You should respect the following train / test split:  
train: first 463,715 examples  
test: last 51,630 examples  
It avoids the 'producer effect' by making sure no song  
from a given artist ends up in both the train and test set.  

## Attribute Information:  
90 attributes, 12 = timbre average, 78 = timbre covariance  
The first value is the year (target), ranging from 1922 to 2011.  
Features extracted from the 'timbre' features from The Echo Nest API.  
We take the average and covariance over all 'segments', each segment  
being described by a 12-dimensional timbre vector.  

## Relevant Papers:
see the website: http://millionsongdataset.com/

In [ ]:
# Load the data from the text file
raw_df = pd.read_csv("./data/YearPredictionMSD.txt", header=None)
years = raw_df[0].to_numpy(dtype=int)

# Drop the year column and assign the attribute data to a numpy ndarray
raw_df.drop(0, axis=1, inplace=True)
attribute_data = raw_df.to_numpy()

In [ ]:
# Split the raw data into training and test sets
DataContainer = namedtuple("DataContainer", ["years", "data"])
TRAIN_IDX = np.arange(0, 463715, dtype=int)
TEST_IDX = np.arange(463715, 463715 + 51630, dtype=int)

# Split data into training and test set, and also separate the year labels
# from the attributes
train = DataContainer(years[TRAIN_IDX], attribute_data[TRAIN_IDX])
test = DataContainer(years[TEST_IDX], attribute_data[TEST_IDX])

# Print the shape of the training and test data to ensure it
# was loaded correctly. Expect 463715/51630 rows, and 90 columns
print(f"Train Data Shape: {train.data.shape}")
print(f"Test Data Shape:  {test.data.shape}")

## Normalization

In [ ]:
# Normalize the training and test data
preprocessing.normalize(train.data, norm='l2', axis=1, copy=False)
preprocessing.normalize(test.data, norm='l2', axis=1, copy=False)

## Dimensionality Reduction

In [ ]:
pca = decomposition.PCA(n_components=25)
pca.fit(train.data)
train_pca = DataContainer(train.years, pca.transform(train.data))
print(f"Training Data PCA Total Explained Variance Ratio = {pca.explained_variance_ratio_.sum()}")

pca = decomposition.PCA(n_components=25)
pca.fit(train.data)
test_pca = DataContainer(test.years, pca.transform(test.data))
print(f"Test Data PCA Total Explained Variance Ratio = {pca.explained_variance_ratio_.sum()}")

# Clustering

In [ ]:
kmeans = cluster.KMeans(n_clusters=8)
kmeans.fit(train.data)

# Results Visualization

In [ ]:
plt.figure()
for i in set(kmeans.labels_):
    x = train.years[kmeans.labels_ == i]
    y = np.random.randn(x.size)
    plt.scatter(x, y)